# IM939 Lab 4 - Part 2 - Crime

Below we work with a subset of the US Census & Crime data. Details about the various variables can be found [here](http://archive.ics.uci.edu/ml/datasets/communities+and+crime).

How can we tackle a dataset with 100 different dimensions? We try using PCA and clustering. There is a lot you can do with this dataset. We encourage you to dig in. Try and find some interesting patterns in the data. Even upload your findings to the Teams channel and discuss.

In [ ]:
import pandas as pd
df = pd.read_csv('censusCrimeClean.csv')

In [ ]:
df

In [ ]:
from sklearn.decomposition import PCA

n_components = 2
 
pca = PCA(n_components=n_components)
df_pca = pca.fit(df.iloc[:, 1:])

In [ ]:
df_pca.explained_variance_

In [ ]:
df_pca_vals = pca.fit_transform(df.iloc[:,1:])
df['c1'] = [item[0] for item in df_pca_vals]
df['c2'] = [item[1] for item in df_pca_vals]

In [ ]:
import seaborn as sns
sns.scatterplot(data = df, x = 'c1', y = 'c2')

Hmm, that looks problematic. What is going on?

We should look at the component loadings. Though there are going to be over 200 of them. We can put them into a Pandas dataframe and then sort them.

In [ ]:
data = {'columns' : df.columns[1:102],
        'component 1' : df_pca.components_[0],
        'component 2' : df_pca.components_[1]}


loadings = pd.DataFrame(data)
loadings.sort_values(by=['component 1'], ascending=False) 

In [ ]:
loadings.sort_values(by=['component 2'], ascending=False) 

The fold variable is messing with our PCA. What does that variable look like.

In [ ]:
df.fold.value_counts()

Aha! It looks to be some sort of categorical variable. Looking into the dataset more, it is actually a variable used for cross tabling.

We should not include this variable in our analysis.

In [ ]:
df.iloc[:, 2:-2]

In [ ]:
from sklearn.decomposition import PCA

n_components = 2

pca_no_fold = PCA(n_components=n_components)
df_pca_no_fold = pca_no_fold.fit(df.iloc[:, 2:-2])

df_pca_vals = pca_no_fold.fit_transform(df.iloc[:, 2:-2])

df['c1_no_fold'] = [item[0] for item in df_pca_vals]
df['c2_no_fold'] = [item[1] for item in df_pca_vals]

sns.scatterplot(data = df, x = 'c1_no_fold', y = 'c2_no_fold')

How do our component loadings look?

In [ ]:
data = {'columns' : df.iloc[:, 2:-4].columns,
        'component 1' : df_pca_no_fold.components_[0],
        'component 2' : df_pca_no_fold.components_[1]}


loadings = pd.DataFrame(data)
loadings_sorted = loadings.sort_values(by=['component 1'], ascending=False)
loadings_sorted.iloc[1:10,:]

In [ ]:
loadings_sorted = loadings.sort_values(by=['component 2'], ascending=False)
loadings_sorted.iloc[1:10,:]

Interesting that our first component variables are income related whereas our second component variables are immigration related. If we look at the projections of the model, coloured by Crime, what do we see?

In [ ]:
sns.scatterplot(data = df,
                x = 'c1_no_fold',
                y = 'c2_no_fold',
                hue = 'ViolentCrimesPerPop',
                size = 'ViolentCrimesPerPop')

What about clustering using these 'income' and 'immigration' components?

In [ ]:
from sklearn.cluster import KMeans

ks = range(1, 10)
inertias = []
for k in ks:
    # Create a KMeans instance with k clusters: model
    model = KMeans(n_clusters=k)
    
    # Fit model to samples
    model.fit(df[['c1_no_fold', 'c2_no_fold']])
    
    # Append the inertia to the list of inertias
    inertias.append(model.inertia_)

import matplotlib.pyplot as plt

plt.plot(ks, inertias, '-o', color='black')
plt.xlabel('number of clusters, k')
plt.ylabel('inertia')
plt.xticks(ks)
plt.show()

Four clusters looks good.

In [ ]:
k_means_4 = KMeans(n_clusters = 3, init = 'random')
k_means_4.fit(df[['c1_no_fold', 'c2_no_fold']])
df['Four clusters'] = pd.Series(k_means_4.predict(df[['c1_no_fold', 'c2_no_fold']].values), index = df.index)

In [ ]:
sns.scatterplot(data = df, x = 'c1_no_fold', y = 'c2_no_fold', hue = 'Four clusters')

Hmm, might we interpret this plot? The plot is unclear. Let us assume c1 is an income component and c2 is an immigration component. Cluster 0 are places high on income and low on immigration. Cluster 2 are low on income and low on immigration. The interesting group are those high on immigration and relatively low on income.

We can include crime on the plot.

In [ ]:
import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = (15,10)
sns.scatterplot(data = df, x = 'c1_no_fold', y = 'c2_no_fold', hue = 'Four clusters', size = 'ViolentCrimesPerPop')

Alas, we stop here in this exercise. You could work on this further. There are outliers you might want to remove. Or, at this stage, you might want to look at more components, 3 or 4 and look for some other factors.